In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - fit_GE_mass_mock.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Fit density profiles to mock APOGEE data
'''

__author__ = "James Lane"

In [ ]:
### Imports

# Basic
import os, sys, pdb, time, tqdm, warnings, multiprocessing, copy, dill as pickle
import numpy as np

# Matplotlib and plotting 
import matplotlib
import matplotlib.pyplot as plt
import corner

# mwdust, isodist stuff
import mwdust
from isodist import FEH2Z, Z2FEH

# Fitting, optimization and statistics
import emcee
import scipy.optimize

# Project specific
sys.path.insert(0,'../../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import iso as piso
from ges_mass import util as putil
from ges_mass import plot as pplot

### Scale parameters
ro = 8.275
vo = 220
zo = 0.0208 # Bennett+ 2019

### Notebook setup

%matplotlib inline
plt.style.use('../../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Preliminaries
Keywords, pathing, loading and setup

In [ ]:
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','NPROCS']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    nprocs = putil.parse_config_dict(cdict,keywords)

In [ ]:
## Pathing
data_dir = base_dir+'data/'
version_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'
ga_dir = data_dir+'gaia_apogee/'+version_dir
gap_dir = data_dir+'gaia_apogee_processed/'+version_dir
ksf_dir = data_dir+'ksf/'+version_dir

In [ ]:
## Filenames
apogee_SF_filename = ga_dir+'apogee_SF.dat'
apogee_effSF_filename = ga_dir+\
    'apogee_effSF_grid_inclArea_z0.001_logAge10.0.dat'
apogee_effSF_mask_filename = ga_dir+\
    'apogee_effSF_grid_mask_z0.001_logAge10.0.npy'
iso_grid_filename = ga_dir+'iso_grid.npy'

In [ ]:
## Loading
# selection function stuff
with open(apogee_SF_filename, 'rb') as f:
    print('\nLoading APOGEE sel. func. from '+apogee_SF_filename)
    apogee_SF = pickle.load(f)
with open(apogee_effSF_filename,'rb') as f:
    print('\nLoading APOGEE eff. sel. func. from '+apogee_effSF_filename)
    apogee_effSF_grid_inclArea = pickle.load(f)
print('\nLoading APOGEE eff. sel. func. mask from '+apogee_effSF_mask_filename)
apogee_effSF_mask = np.load(apogee_effSF_mask_filename)

# dust map
dmap = mwdust.Combined19(filter='2MASS H') # dustmap from mwdust, use most recent

# isochrone
print('\nLoading isochrone grid from '+iso_grid_filename)
iso_grid = np.load(iso_grid_filename)
z = 0.0010 # Normal parameters for single isochrone
log_age = 10.0
iso = iso_grid[(iso_grid['Zini']==z) & (iso_grid['logAge']==log_age)]

In [ ]:
## Effective selection function grid
# Distance modulus grid
dmods,ds = putil.make_dmod_grid(ndmod,dmod_min,dmod_max)

# Grid of positions in the APOGEE effective selection function grid
Rgrid,phigrid,zgrid = pmass.Rphizgrid(apogee_SF,dmods,ro=ro,zo=zo)

# Apply the effective selection function grid mask
apof = apogee_effSF_grid_inclArea[apogee_effSF_mask]
Rgrid = Rgrid[apogee_effSF_mask]
phigrid = phigrid[apogee_effSF_mask]
zgrid = zgrid[apogee_effSF_mask]

# Include the distance modulus Jacobian
Jac_dmod = ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])
Jac_rad = (np.pi/180.)**2.
apof = apof * Jac_dmod * Jac_rad

# JKmins
jkmins = np.array([apogee_SF.JKmin(apogee_SF._locations[i]) \
                   for i in range(len(apogee_SF._locations))])
jkmins = jkmins[apogee_effSF_mask]

### Load mock data

In [ ]:
# Filenames
mock_number = '40'
mock_path = '../data/mock_'+mock_number+'/'
allstar_filename = mock_path+'/allstar.npy'
orbs_filename = mock_path+'/orbs.pkl'
data_omask_filename = mock_path+'/omask.npy'

# Load
with open(mock_path+'orbs.pkl','rb') as f:
    orbs_nomask = pickle.load(f)
allstar_nomask = np.load(allstar_filename)
data_mask = np.load(data_omask_filename)

# Setup data array and mask the data
orbs = orbs_nomask[data_mask]
allstar = allstar_nomask[data_mask]
mrpz = np.array([orbs.R(use_physical=True).value,
                 orbs.phi(use_physical=True).value,
                 orbs.z(use_physical=True).value]).T

### Do the fitting

In [ ]:
# Definitions
densfunc = pdens.triaxial_single_angle_zvecpa
# alpha, p, q, eta, theta, pa
init = np.array([2.0, 0.5, 0.5, 0.5, 0.5, 0.5])
ndim = len(init)
nwalkers = 200
nit = 200
ncut = 100
Rdata,phidata,zdata = mrpz.T
effsel = copy.deepcopy(apof)

# Maximum likelihood
opt = scipy.optimize.fmin(lambda x: pmass.mloglike(x, densfunc, effsel, 
    Rgrid, phigrid, zgrid, Rdata, phidata, zdata), init, 
    full_output=True)
print(opt[0])

#pos = [init + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]
pos = [opt[0] + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]

with multiprocessing.Pool(nprocs) as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, pmass.loglike, 
        args=(densfunc, effsel, Rgrid, phigrid, zgrid, Rdata, phidata, zdata), 
        pool=pool)
    print('Generating MCMC samples...')
    for i, result in enumerate(sampler.sample(pos, iterations=nit)):
        if (i+1)%10 == 0: print('sampled '+str(i+1)+'/'+str(nit))
        continue
    # Flatten the ensemble of walkers to a set of samples, remove ncut from each
    # samples = sampler.chain[:, ncut:, :].reshape((-1, ndim))
    samples = sampler.get_chain(flat=True,discard=ncut)

In [ ]:
labels = [r'$\alpha$', r'$p$', r'$q$', r'$\theta$', r'$\eta$', r'$\phi$']
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=[3.5,0.5,1.,0.,1.,1./6.], truth_color='Red', 
                    labels=labels)
fig.show()

### Plot the shape of the fitted profile, as well as the expected profile

In [ ]:
# Expected
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    params = [3.5, 0.5, 1.0, 0.0, 1.0, 1./6.]
    fig,axs = pplot.plot_density_xyz(densfunc,params)
    fig.show()

In [ ]:
# Actual
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    params_fit = np.median(samples,axis=0)
    fig,axs = pplot.plot_density_xyz(densfunc,params_fit)
    fig.show()

### Try changing `ro` to 8 and `zo` to 0 for the eSSF grid

In [ ]:
# Grid of positions in the APOGEE effective selection function grid
Rgrid,phigrid,zgrid = pmass.Rphizgrid(apogee_SF,dmods,ro=8,zo=0)

# Apply the effective selection function grid mask
apof = apogee_effSF_grid_inclArea[apogee_effSF_mask]
apof *= ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])*(np.pi/180.)**2.
Rgrid = Rgrid[apogee_effSF_mask]
phigrid = phigrid[apogee_effSF_mask]
zgrid = zgrid[apogee_effSF_mask]

# Definitions
densfunc = pdens.triaxial_single_angle_zvecpa
# alpha, p, q, eta, theta, pa
init = np.array([2.0, 0.5, 0.5, 0.5, 0.5, 0.5])
ndim = len(init)
nwalkers = 200
nit = 200
ncut = 100
Rdata,phidata,zdata = mrpz.T
effsel = copy.deepcopy(apof)

# Maximum likelihood
opt = scipy.optimize.fmin(lambda x: pmass.mloglike(x, densfunc, effsel, 
    Rgrid, phigrid, zgrid, Rdata, phidata, zdata), init, 
    full_output=True)
print(opt[0])

#pos = [init + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]
pos = [opt[0] + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]

with multiprocessing.Pool(nprocs) as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, pmass.loglike, 
        args=(densfunc, effsel, Rgrid, phigrid, zgrid, Rdata, phidata, zdata), 
        pool=pool)
    print('Generating MCMC samples...')
    for i, result in enumerate(sampler.sample(pos, iterations=nit)):
        if (i+1)%10 == 0: print('sampled '+str(i+1)+'/'+str(nit))
        continue
    # Flatten the ensemble of walkers to a set of samples, remove ncut from each
    # samples = sampler.chain[:, ncut:, :].reshape((-1, ndim))
    samples = sampler.get_chain(flat=True,discard=ncut)

labels = [r'$\alpha$', r'$p$', r'$q$', r'$\theta$', r'$\eta$', r'$\phi$']
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=[3.5,0.5,1.,0.,1.,1./6.], truth_color='Red', 
                    labels=labels)
fig.show()

# Grid of positions in the APOGEE effective selection function grid
Rgrid,phigrid,zgrid = pmass.Rphizgrid(apogee_SF,dmods,ro=ro,zo=zo)

# Apply the effective selection function grid mask
apof = apogee_effSF_grid_inclArea[apogee_effSF_mask]
apof *= ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])*(np.pi/180.)**2.
Rgrid = Rgrid[apogee_effSF_mask]
phigrid = phigrid[apogee_effSF_mask]
zgrid = zgrid[apogee_effSF_mask]

### Try altering the starting point for maximum likelihood

In [ ]:
# Definitions
densfunc = pdens.triaxial_single_angle_zvecpa
# alpha, p, q, theta, eta, pa
init = np.array([3.5, 0.5, 1.0, 0.0, 1.0, 1./6.])
ndim = len(init)
nwalkers = 200
nit = 200
ncut = 100
Rdata,phidata,zdata = mrpz.T
effsel = copy.deepcopy(apof)

# Maximum likelihood
opt = scipy.optimize.fmin(lambda x: pmass.mloglike(x, densfunc, effsel, 
    Rgrid, phigrid, zgrid, Rdata, phidata, zdata), init, 
    full_output=True)
print(opt[0])

#pos = [init + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]
pos = [opt[0] + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]

t1 = time.time()
with multiprocessing.Pool(nprocs) as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, pmass.loglike, 
        args=(densfunc, effsel, Rgrid, phigrid, zgrid, Rdata, phidata, zdata), 
        pool=pool)
    print('Generating MCMC samples...')
    for i, result in enumerate(sampler.sample(pos, iterations=nit)):
        if (i+1)%10 == 0: print('sampled '+str(i+1)+'/'+str(nit))
        continue
    # Flatten the ensemble of walkers to a set of samples, remove ncut from each
    # samples = sampler.chain[:, ncut:, :].reshape((-1, ndim))
    samples = sampler.get_chain(flat=True,discard=ncut)
t2 = time.time()

In [ ]:
labels = [r'$\alpha$', r'$p$', r'$q$', r'$\theta$', r'$\eta$', r'$\phi$']
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=[3.5,0.5,1.,0.,1.,1./6.], truth_color='Red', 
                    labels=labels)
fig.show()

In [ ]:
# Expected
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    params = [3.5, 0.5, 1.0, 0.0, 1.0, 1./6.]
    fig,axs = pplot.plot_density_xyz(densfunc,params)
    fig.show()

In [ ]:
# Actual
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    params_fit = np.median(samples,axis=0)
    fig,axs = pplot.plot_density_xyz(densfunc,params_fit)
    fig.show()

### Change the starting position, but also try changing `ro` to 8 and `zo` to 0 for the eSSF grid

In [ ]:
# Grid of positions in the APOGEE effective selection function grid
Rgrid,phigrid,zgrid = pmass.Rphizgrid(apogee_SF,dmods,ro=8,zo=0)

# Apply the effective selection function grid mask
apof = apogee_effSF_grid_inclArea[apogee_effSF_mask]
apof *= ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])*(np.pi/180.)**2.
Rgrid = Rgrid[apogee_effSF_mask]
phigrid = phigrid[apogee_effSF_mask]
zgrid = zgrid[apogee_effSF_mask]

# Definitions
densfunc = pdens.triaxial_single_angle_zvecpa
# alpha, p, q, eta, theta, pa
init = np.array([2.0, 0.5, 0.5, 0.5, 0.5, 0.5])
ndim = len(init)
nwalkers = 200
nit = 200
ncut = 100
Rdata,phidata,zdata = mrpz.T
effsel = copy.deepcopy(apof)

# Maximum likelihood
opt = scipy.optimize.fmin(lambda x: pmass.mloglike(x, densfunc, effsel, 
    Rgrid, phigrid, zgrid, Rdata, phidata, zdata), init, 
    full_output=True)
print(opt[0])

#pos = [init + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]
pos = [opt[0] + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]

with multiprocessing.Pool(nprocs) as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, pmass.loglike, 
        args=(densfunc, effsel, Rgrid, phigrid, zgrid, Rdata, phidata, zdata), 
        pool=pool)
    print('Generating MCMC samples...')
    for i, result in enumerate(sampler.sample(pos, iterations=nit)):
        if (i+1)%10 == 0: print('sampled '+str(i+1)+'/'+str(nit))
        continue
    # Flatten the ensemble of walkers to a set of samples, remove ncut from each
    # samples = sampler.chain[:, ncut:, :].reshape((-1, ndim))
    samples = sampler.get_chain(flat=True,discard=ncut)

labels = [r'$\alpha$', r'$p$', r'$q$', r'$\theta$', r'$\eta$', r'$\phi$']
fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    truths=[3.5,0.5,1.,0.,1.,1./6.], truth_color='Red', 
                    labels=labels)
fig.show()

# Grid of positions in the APOGEE effective selection function grid
Rgrid,phigrid,zgrid = pmass.Rphizgrid(apogee_SF,dmods,ro=ro,zo=zo)

# Apply the effective selection function grid mask
apof = apogee_effSF_grid_inclArea[apogee_effSF_mask]
apof *= ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])*(np.pi/180.)**2.
Rgrid = Rgrid[apogee_effSF_mask]
phigrid = phigrid[apogee_effSF_mask]
zgrid = zgrid[apogee_effSF_mask]

### Try and change the portion of the chain selected for drawing samples

In [ ]:
# Definitions
densfunc = pdens.triaxial_single_angle_zvecpa
# alpha, p, q, theta, eta, pa
init = np.array([3.5, 0.5, 1.0, 0.0, 1.0, 1./6.])
ndim = len(init)
nwalkers = 200
nit = 1000
ncut = 100
Rdata,phidata,zdata = mrpz.T
effsel = copy.deepcopy(apof)

# Maximum likelihood
opt = scipy.optimize.fmin(lambda x: pmass.mloglike(x, densfunc, effsel, 
    Rgrid, phigrid, zgrid, Rdata, phidata, zdata), init, 
    full_output=True)
print(opt[0])

#pos = [init + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]
pos = [opt[0] + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]

t1 = time.time()
with multiprocessing.Pool(nprocs) as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, pmass.loglike, 
        args=(densfunc, effsel, Rgrid, phigrid, zgrid, Rdata, phidata, zdata), 
        pool=pool)
    print('Generating MCMC samples...')
    for i, result in enumerate(sampler.sample(pos, iterations=nit)):
        if (i+1)%10 == 0: print('sampled '+str(i+1)+'/'+str(nit))
        continue
    # Flatten the ensemble of walkers to a set of samples, remove ncut from each
    # samples = sampler.chain[:, ncut:, :].reshape((-1, ndim))
t2 = time.time()
print('MCMC took '+str(t2-t1)+' s')

In [ ]:
chain_cuts = np.arange(nit,step=100,dtype=int)
tau = np.zeros((len(chain_cuts),6))
for i in range(len(chain_cuts)):
    tau[i] = sampler.get_autocorr_time(discard=chain_cuts[i],quiet=True)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
for i in range(6):
    ax.plot(chain_cuts,tau[:,i]/(nit-chain_cuts))
ax.axhline(1/50, color='Black', linestyle='dashed')
ax.set_xlabel('Samples cut from chain (length '+str(nit)+')')
ax.set_ylabel(r'$\tau/N$')

In [ ]:
labels = [r'$\alpha$', r'$p$', r'$q$', r'$\theta$', r'$\eta$', r'$\phi$']
chain_range = [[0,100],[50,150],[100,200],[150,250],[200,300],[250,350],[300,400]]
for i in range(len(chain_range)):
    nmin,nmax = chain_range[i]
    samples = sampler.chain[:, nmin:nmax, :].reshape((-1, ndim))
    fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                        truths=[3.5,0.5,1.,0.,1.,1./6.], truth_color='Red', 
                        labels=labels)
    fig.suptitle('Using samples '+str(nmin)+':'+str(nmax)+'/400')
    fig.show()

### Test many profiles

In [ ]:
mock_numbers = ['40','41','42','43','44','45']

# [alpha,b,c,theta,eta,pa]
# theta normalized 0 to 2pi
# eta normalized -1 to 1
# pa normalized 0 to pi
params_correct = [
    [3.5, 0.5, 1.0, 0.0      , 1.0  , np.pi/6],
    [3.5, 0.5, 0.8, 0.0      , 1.0  , np.pi/5],
    [3.5, 1.0, 0.5, 0.0      , 1.0  , 0.0    ],
    [3.5, 0.5, 1.0, 0.0      , 0.0  , 0.0    ],
    [3.5, 0.5, 1.0, np.pi/2. , 0.0  , 0.0    ],
    [3.5, 0.5, 1.0, 0.0      , 0.707, np.pi/6]
]

densfunc = pdens.triaxial_single_angle_zvecpa

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for k in range(len(mock_numbers)):
        
        #if k > 2: continue
        
        # Load data
        mock_number = mock_numbers[k]
        mock_path = '../data/mock_'+mock_number+'/'
        allstar_filename = mock_path+'/allstar.npy'
        orbs_filename = mock_path+'/orbs.pkl'
        data_omask_filename = mock_path+'/omask.npy'
        effsel = copy.deepcopy(apof)

        # Load
        with open(mock_path+'orbs.pkl','rb') as f:
            orbs_nomask = pickle.load(f)
        allstar_nomask = np.load(allstar_filename)
        data_mask = np.load(data_omask_filename)

        # Setup data array and mask the data
        orbs = orbs_nomask[data_mask]
        allstar = allstar_nomask[data_mask]
        Rdata = orbs.R(use_physical=True).value
        phidata = orbs.phi(use_physical=True).value
        zdata = orbs.z(use_physical=True).value
        
        # alpha, p, q, eta, theta, pa
        params = pdens.normalize_parameters(params_correct[k],densfunc)
        init = np.asarray(params)
        ndim = len(init)
        nwalkers = 25
        nit = 500
        ncut = 200
        
        # Maximum likelihood
#         opt = scipy.optimize.fmin(lambda x: pmass.mloglike(x, densfunc, effsel, 
#             Rgrid, phigrid, zgrid, Rdata, phidata, zdata), init, 
#             full_output=True)
#         print(opt[0])

        pos = [init + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]
        #pos = [opt[0] + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]
    
        print('Sampling '+str(k))
        with multiprocessing.Pool(nprocs) as pool:
            sampler = emcee.EnsembleSampler(nwalkers, ndim, pmass.loglike, 
                args=(densfunc, effsel, Rgrid, phigrid, zgrid, Rdata, phidata, zdata), 
                pool=pool)
            #print('Generating MCMC samples...')
            for i, result in enumerate(sampler.sample(pos, iterations=nit)):
                if (i+1)%10 == 0: pass# print('sampled '+str(i+1)+'/'+str(nit))
                continue
            # samples = sampler.chain[:, ncut:, :].reshape((-1, ndim))
            samples = sampler.get_chain(flat=True,discard=ncut)
        
        print('Plotting '+str(k))
        labels = [r'$\alpha$', r'$p$', r'$q$', r'$\theta$', r'$\eta$', r'$\phi$']
        fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                            truths=init, truth_color='Red', 
                            labels=labels)
        fig.show()
        
        fig,axs = pplot.plot_density_xyz(densfunc,np.median(samples,axis=0))
        fig.suptitle('Fit')
        fig.show()
        
        fig,axs = pplot.plot_density_xyz(densfunc,init)
        fig.suptitle('Actual')
        fig.show()

In [ ]:
mock_numbers = ['40','41','42','43','44','45']

# [alpha,b,c,theta,eta,pa]
# theta normalized 0 to 2pi
# eta normalized -1 to 1
# pa normalized 0 to pi
params_correct = [
    [3.5, 0.5, 1.0, 0.0      , 1.0  , np.pi/6],
    [3.5, 0.5, 0.8, 0.0      , 1.0  , np.pi/5],
    [3.5, 1.0, 0.5, 0.0      , 1.0  , 0.0    ],
    [3.5, 0.5, 1.0, 0.0      , 0.0  , 0.0    ],
    [3.5, 0.5, 1.0, np.pi/2. , 0.0  , 0.0    ],
    [3.5, 0.5, 1.0, 0.0      , 0.707, np.pi/6]
]

densfunc = pdens.triaxial_single_angle_zvecpa

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for k in range(len(mock_numbers)):
        
        #if k > 2: continue
        
        # Load data
        mock_number = mock_numbers[k]
        mock_path = '../data/mock_'+mock_number+'/'
        allstar_filename = mock_path+'/allstar.npy'
        orbs_filename = mock_path+'/orbs.pkl'
        data_omask_filename = mock_path+'/omask.npy'
        effsel = copy.deepcopy(apof)

        # Load
        with open(mock_path+'orbs.pkl','rb') as f:
            orbs_nomask = pickle.load(f)
        allstar_nomask = np.load(allstar_filename)
        data_mask = np.load(data_omask_filename)

        # Setup data array and mask the data
        orbs = orbs_nomask[data_mask]
        allstar = allstar_nomask[data_mask]
        Rdata = orbs.R(use_physical=True).value
        phidata = orbs.phi(use_physical=True).value
        zdata = orbs.z(use_physical=True).value
        
        # alpha, p, q, eta, theta, pa
        params = pdens.normalize_parameters(params_correct[k],densfunc)
        init = np.random.random(6)
        init[0] = init[0]*5
        ndim = len(init)
        nwalkers = 100
        nit = 1000
        ncut = 200
        
        # Maximum likelihood
        opt = scipy.optimize.fmin(lambda x: pmass.mloglike(x, densfunc, effsel, 
            Rgrid, phigrid, zgrid, Rdata, phidata, zdata), init, 
            full_output=True)
        print(opt[0])

        #pos = [init + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]
        pos = [opt[0] + 1e-3*np.random.randn(ndim) for i in range(nwalkers)]
    
        print('Sampling '+str(k))
        with multiprocessing.Pool(nprocs) as pool:
            sampler = emcee.EnsembleSampler(nwalkers, ndim, pmass.loglike, 
                args=(densfunc, effsel, Rgrid, phigrid, zgrid, Rdata, phidata, zdata), 
                pool=pool)
            #print('Generating MCMC samples...')
            for i, result in enumerate(sampler.sample(pos, iterations=nit)):
                if (i+1)%10 == 0: pass# print('sampled '+str(i+1)+'/'+str(nit))
                continue
            # samples = sampler.chain[:, ncut:, :].reshape((-1, ndim))
            samples = sampler.get_chain(flat=True,discard=ncut)
        
        print('Plotting '+str(k))
        labels = [r'$\alpha$', r'$p$', r'$q$', r'$\theta$', r'$\eta$', r'$\phi$']
        fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                            truths=params, truth_color='Red', 
                            labels=labels)
        fig.show()
        
        fig,axs = pplot.plot_density_xyz(densfunc,np.median(samples,axis=0))
        fig.suptitle('Fit')
        fig.show()
        
        fig,axs = pplot.plot_density_xyz(densfunc,np.asarray(params))
        fig.suptitle('Actual')
        fig.show()